## Tutorial


### Step 1: Reclassify the existing CLM base layer that represents CLM PFTs to Demeter's desired output land classes


#### Import the class we need:


In [17]:
from clm_demeter.reclassify_base import ReclassBaselayer


#### Check out help on our `ReclassBaselayer` class to see what it requires:

In [18]:
help(ReclassBaselayer)


Help on class ReclassBaselayer in module clm_demeter.reclassify_base:

class ReclassBaselayer(builtins.object)
 |  Reclassify CLM PFTs in the baselayer for Demeter to Demeter final landcover classes.
 |  
 |  :param clm_baselayer_file:              Full path with file name and extension to
 |                                          the CLM baselayer prepared for Demeter
 |                                          in units square-degrees.
 |  
 |  :param clm_spatial_allocation_file:     Full path with file name and extension to
 |                                          the Demeter spatial allocation file for CLM
 |                                          landclasses.
 |  
 |  :param out_clm_baselayer_file:          Full path with file name and extension to
 |                                          to save the reclassified baselayer.
 |  
 |  :method demeter_to_clm_map:             Create a dictionary of Demeter final landclasses
 |                                          to CLM P

#### Set the paths to our files including the desired output:


In [23]:
clm_baselayer_file = '/Users/d3y010/projects/demeter/clm/baselayerdata.csv'
clm_spatial_allocation_file = '/Users/d3y010/projects/demeter/clm/spatial_allocation_rules.csv'
out_clm_baselayer_file = '/Users/d3y010/projects/demeter/clm/clm_baselayer_by_final_lcs.csv'


#### Generate a reclassififed base layer for Demeter from the original CLM base layer:


In [24]:
base = ReclassBaselayer(clm_baselayer_file, clm_spatial_allocation_file, out_clm_baselayer_file)#%% md


#### To take a look at the resulting data frame to 5 rows:

In [25]:
base.df_clm_baselayer.head()


,Latcoord,Loncoord,FID,regAEZ,region_id,aez_id,OBJECTID,Barren,NET_tem,NET_bor,...,Palm_rf,Palm_irr,Fodder_rf,Fodder_irr,Rice_rf,Rice_irr,Sugarcrop_rf,Sugarcrop_irr,Bioenergy_rf,Bioenergy_irr
0,71.525,-179.975,370,2313,23,13,1,0.000550,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,71.475,-179.975,371,2313,23,13,2,0.000675,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,71.425,-179.975,372,2313,23,13,3,0.000675,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,71.375,-179.975,373,2313,23,13,4,0.000575,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,71.325,-179.975,374,2313,23,13,5,0.000625,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


### Step 2: Disaggregate any projected land class areas per subregion by what is fractionally represented in the base layer.  This prevents Demeter's built-in method from spliting the area evenly regardless of what is in the observed data.


#### Import the batch processing function we need:


In [26]:
import clm_demeter.reclassify_projected as rp


#### Check out the help on the batch function as well as the class that does the work:


In [28]:
help(rp.batch_process_split)


Help on function batch_process_split in module clm_demeter.reclassify_projected:

batch_process_split(projected_allocation_file, observed_baselayer_file, projected_file, out_projected_file, metric, gcam_year_list)
    Batch process projected land class disaggregation.
    
    :param projected_allocation_file:   Full path with file name and extension of the projected allocation file from
                                        Demeter that maps GCAM land classes to the final land cover classes in
                                        Demeter.
    
    :param observed_file:               Full path with file name and extension of the observed data file to be used in the
                                        Demeter run.
    
    :param projected_file:              Full path with file name and extension of the projected data file that has been
                                        extracted from a GCAM output database for use with Demeter.
    
    :param out_projected_file:        

In [29]:
help(rp.GcamLandclassSplit)


Help on class GcamLandclassSplit in module clm_demeter.reclassify_projected:

class GcamLandclassSplit(builtins.object)
 |  Split a GCAM landclass into multiple classes based on the fractional amount present in the observed data per
 |  subregion.  This method is more desirable than the default "even percentage" split that Demeter conducts.  The
 |  output file replaces the GCAM target landclass (e.g. RockIceDesert) with the user-selected classes (e.g. snow and
 |  sparse) per subregion.  The new file becomes what is referenced as the projected file in Demeter.
 |  
 |  :param observed_file:               Full path with file name and extension of the observed data file to be used in the
 |                                      Demeter run.
 |  
 |  :param projected_file:              Full path with file name and extension of the projected data file that has been
 |                                      extracted from a GCAM output database for use with Demeter.
 |  
 |  :param target_lan

#### Set the paths to our files including the desired output; let's use the output base layer from above:


In [30]:
projected_allocation_file = '/Users/d3y010/projects/demeter/clm/gcam_allocation_rules.csv'
projected_file = '/Users/d3y010/projects/demeter/clm/gcam_ssp4_34_lumip.csv'
out_projected_file = '/Users/d3y010/projects/demeter/clm/gcam_ssp4_34_lumip_disagg.csv'
metric = 'aez_id'
gcam_year_list = [1990] + list(range(2005, 2105, 5))


#### Run batch disaggregation of land classes:


In [32]:
proj = rp.batch_process_split(projected_allocation_file, 
                                out_clm_baselayer_file,
                                projected_file,
                                out_projected_file,
                                metric,
                                gcam_year_list)

Disaggregating projected land class 'biomass' to '['Bioenergy_rf', 'Bioenergy_irr']'
Disaggregating projected land class 'Corn' to '['Corn_rf', 'Corn_irr']'
Disaggregating projected land class 'FiberCrop' to '['Cotton_rf', 'Cotton_irr']'
Disaggregating projected land class 'FodderGrass' to '['Fodder_rf', 'Fodder_irr']'
Disaggregating projected land class 'FodderHerb' to '['Fodder_rf', 'Fodder_irr']'
Disaggregating projected land class 'Forest' to '['NET_tem', 'NET_bor', 'NDT_bor', 'BET_tro', 'BET_tem', 'BDT_tro', 'BDT_tem', 'BDT_bor']'
Disaggregating projected land class 'Grassland' to '['C3_gra_arc', 'C3_gra', 'C4_gra']'
Disaggregating projected land class 'MiscCrop' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land class 'OilCrop' to '['Soy_rf', 'Soy_irr']'
Disaggregating projected land class 'OtherArableLand' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land class 'OtherGrain' to '['OtherCrop_rf', 'OtherCrop_irr']'
Disaggregating projected land 

In [33]:
#### Take a look at the resulting data frame to 5 rows:
proj.head()

,region,region_id,metric_id,landclass,Units,1990,2005,2010,2015,2020,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,Africa_Eastern,2,1,BDS_bor,thous km2,0.035590,0.039828,0.039904,0.039074,0.037952,...,0.022381,0.019151,0.015931,0.012804,0.009982,0.007677,0.005844,0.004404,0.003245,0.002363
1,Africa_Eastern,2,1,BDS_tem,thous km2,292.910145,327.793526,328.418034,321.587043,312.351053,...,184.197236,157.619516,131.114045,105.378570,82.151223,63.185178,48.097220,36.245917,26.705491,19.452092
2,Africa_Eastern,2,1,BDT_bor,thous km2,0.000252,0.000067,0.000101,0.000102,0.000103,...,0.000104,0.000099,0.000092,0.000084,0.000076,0.000069,0.000064,0.000058,0.000053,0.000047
3,Africa_Eastern,2,1,BDT_tem,thous km2,0.037943,0.010136,0.015171,0.015332,0.015528,...,0.015726,0.014857,0.013800,0.012623,0.011472,0.010442,0.009580,0.008767,0.007911,0.007116
4,Africa_Eastern,2,1,BDT_tro,thous km2,2.663521,0.711554,1.064938,1.076285,1.090025,...,1.103949,1.042895,0.968740,0.886096,0.805303,0.732982,0.672478,0.615391,0.555352,0.499502
